In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Flatten
from keras.callbacks import TensorBoard
import keras_tuner

In [13]:
x_train.shape

(404, 13)

In [9]:
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data()

57026/57026 [==============================] - 0s 1us/step


In [12]:
x_train=x_train/255
x_test=x_test/255

In [15]:
# Training a simple network
model=Sequential()
model.add(Dense(10,activation="relu",input_dim=13))
model.add(Dense(1,activation="linear"))
model.compile(optimizer="Adam",loss="mae",metrics=["mae"])

In [17]:
model.fit(x_train,y_train,epochs=10,validation_split=0.2)

Epoch 1/10
11/11 [==============================] - 2s 40ms/step - loss: 23.2999 - mae: 23.2999 - val_loss: 24.7949 - val_mae: 24.7949
Epoch 2/10
11/11 [==============================] - 0s 43ms/step - loss: 23.1832 - mae: 23.1832 - val_loss: 24.6812 - val_mae: 24.6812
Epoch 3/10
11/11 [==============================] - 0s 23ms/step - loss: 23.0717 - mae: 23.0717 - val_loss: 24.5734 - val_mae: 24.5734
Epoch 4/10
11/11 [==============================] - 0s 20ms/step - loss: 22.9662 - mae: 22.9662 - val_loss: 24.4718 - val_mae: 24.4718
Epoch 5/10
11/11 [==============================] - 0s 20ms/step - loss: 22.8667 - mae: 22.8667 - val_loss: 24.3753 - val_mae: 24.3753
Epoch 6/10
11/11 [==============================] - 0s 24ms/step - loss: 22.7717 - mae: 22.7717 - val_loss: 24.2834 - val_mae: 24.2834
Epoch 7/10
11/11 [==============================] - 0s 20ms/step - loss: 22.6814 - mae: 22.6814 - val_loss: 24.1954 - val_mae: 24.1954
Epoch 8/10
11/11 [==============================] - 0s 

In [49]:
def models1(par):
    model=Sequential()
    model.add(Flatten(input_shape=(28,28)))
    model.add(Dense(par.Int("nu_neuron",1,100),activation=par.Choice("activatio_c",values=["sigmoid","tanh","relu"])))
    model.add(Dropout(par.Choice("dprate",values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(10,activation="softmax"))
    model.compile(optimizer=par.Choice("opt",values=["sgd","rmsprop"]),loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    return model

In [53]:
robj=keras_tuner.RandomSearch(models1,objective="val_accuracy",max_trials=5,directory=r"C:\Users\LENOVO\Pictures\logs6")

In [54]:
robj.search(x_train,y_train,epochs=2,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.9484999775886536

Best val_accuracy So Far: 0.9610000252723694
Total elapsed time: 00h 03m 44s
INFO:tensorflow:Oracle triggered exit


In [55]:
robj.get_best_hyperparameters()[0].values

{'nu_neuron': 77, 'activatio_c': 'tanh', 'opt': 'rmsprop'}

In [56]:
finalmodel=robj.get_best_models(num_models=1)[0]

In [57]:
finalmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 77)                60445     
                                                                 
 dense_1 (Dense)             (None, 10)                780       
                                                                 
Total params: 61,225
Trainable params: 61,225
Non-trainable params: 0
_________________________________________________________________


In [58]:
finalmodel.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [59]:
finalmodel.fit(x_train,y_train,epochs=50,batch_size=100,validation_split=0.3)

Epoch 1/50
420/420 [==============================] - 6s 12ms/step - loss: 0.1198 - accuracy: 0.9655 - val_loss: 0.1160 - val_accuracy: 0.9674
Epoch 2/50
420/420 [==============================] - 8s 18ms/step - loss: 0.0996 - accuracy: 0.9722 - val_loss: 0.1114 - val_accuracy: 0.9677
Epoch 3/50
420/420 [==============================] - 6s 14ms/step - loss: 0.0855 - accuracy: 0.9769 - val_loss: 0.1055 - val_accuracy: 0.9696
Epoch 4/50
420/420 [==============================] - 6s 14ms/step - loss: 0.0735 - accuracy: 0.9801 - val_loss: 0.1058 - val_accuracy: 0.9688
Epoch 5/50
420/420 [==============================] - 6s 14ms/step - loss: 0.0645 - accuracy: 0.9828 - val_loss: 0.1002 - val_accuracy: 0.9702
Epoch 6/50
420/420 [==============================] - 6s 13ms/step - loss: 0.0561 - accuracy: 0.9859 - val_loss: 0.1001 - val_accuracy: 0.9706
Epoch 7/50
420/420 [==============================] - 6s 14ms/step - loss: 0.0490 - accuracy: 0.9879 - val_loss: 0.1009 - val_accuracy: 0.9698

In [45]:
x_test.shape

(10000, 28, 28)

In [4]:
def models1(par):
    model=Sequential()
    model.add(Flatten(input_shape=(28,28,)))
    
    x=0
    for y in range(par.Int("hiddenlayers",2,100)):
        model.add(Dense(par.Int("neuron{}".format(x),2,50),activation=par.Choice("ac",value=["relu","sigmoid"]))
        x+=1
        
    
    model.add(Dense(10,activation="softmax"))
    model.compile(optimizer=par.Choice("opt",values=["sgd","rmsprop"]),loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    return model

In [5]:
robj=keras_tuner.Hyperband(models1,objective="val_accuracy",max_trials=7,directory=r"C:\Users\LENOVO\Pictures\logs50")

In [6]:
robj.search(x_train,y_train,epochs=30,validation_data=(x_test,y_test))

Trial 7 Complete [00h 03m 13s]
val_accuracy: 0.11349999904632568

Best val_accuracy So Far: 0.11349999904632568
Total elapsed time: 00h 18m 53s
INFO:tensorflow:Oracle triggered exit


In [7]:
finalmodel=robj.get_best_models(num_models=1)[0]

In [8]:
finalmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 26)                20410     
                                                                 
 dense_1 (Dense)             (None, 4)                 108       
                                                                 
 dense_2 (Dense)             (None, 2)                 10        
                                                                 
 dense_3 (Dense)             (None, 2)                 6         
                                                                 
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 2)                 6

In [ ]:
model.fit(x_train,y_train,epochs=200)